In [38]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import push_notebook, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, FactorRange, Whisker
from bokeh.models.glyphs import VArea
from bokeh.models.tools import HoverTool
import bokeh.palettes
from bokeh.transform import factor_cmap
import math 
import pandas as pd

df = pd.read_csv('Results/linpack_repeat35.csv')
print(df)


     runtime image_name ressource_limit  repeat  gflops  \
0    default    linpack         default       1  46.923   
1    default    linpack         default      10  46.900   
2    default    linpack         default      11  46.870   
3    default    linpack         default      12  46.979   
4    default    linpack         default      13  46.868   
..       ...        ...             ...     ...     ...   
309   katafc    linpack             min       5  11.998   
310   katafc    linpack             min       6  12.027   
311   katafc    linpack             min       7  12.006   
312   katafc    linpack             min       8  12.002   
313   katafc    linpack             min       9  12.019   

                                              filepath  
0    /opt/auswertung/Raw/bench_linpack_repeat35/def...  
1    /opt/auswertung/Raw/bench_linpack_repeat35/def...  
2    /opt/auswertung/Raw/bench_linpack_repeat35/def...  
3    /opt/auswertung/Raw/bench_linpack_repeat35/def...  
4    /

In [39]:
filterd = df
# Remove -1 Values
filterd = filterd[(filterd.gflops != -1)]
print(filterd)

grouped = filterd

print(grouped.to_string())

     runtime image_name ressource_limit  repeat  gflops  \
0    default    linpack         default       1  46.923   
1    default    linpack         default      10  46.900   
2    default    linpack         default      11  46.870   
3    default    linpack         default      12  46.979   
4    default    linpack         default      13  46.868   
..       ...        ...             ...     ...     ...   
309   katafc    linpack             min       5  11.998   
310   katafc    linpack             min       6  12.027   
311   katafc    linpack             min       7  12.006   
312   katafc    linpack             min       8  12.002   
313   katafc    linpack             min       9  12.019   

                                              filepath  
0    /opt/auswertung/Raw/bench_linpack_repeat35/def...  
1    /opt/auswertung/Raw/bench_linpack_repeat35/def...  
2    /opt/auswertung/Raw/bench_linpack_repeat35/def...  
3    /opt/auswertung/Raw/bench_linpack_repeat35/def...  
4    /

In [40]:
# Fehlerrechnung

std = grouped.groupby(['runtime', 'ressource_limit'])['gflops'].std().reset_index(name='std')
print(std)

mean = grouped.groupby(['runtime', 'ressource_limit'])['gflops'].mean().reset_index(name='mean')
print(mean)

std['mean'] = mean[['mean']]

grouped_std_mean = std
print(grouped_std_mean)

# 95% Vertrauensintervall
grouped_std_mean['abweichung_in_prozent'] = grouped_std_mean['std']  / math.sqrt(35) * 1.6909 * 100 / grouped_std_mean['mean']
grouped_std_mean['konf_intervall_unten'] = grouped_std_mean['mean'] - grouped_std_mean['std']  / math.sqrt(35) * 1.6909
grouped_std_mean['konf_intervall_oben'] = grouped_std_mean['mean'] + grouped_std_mean['std']  / math.sqrt(35) * 1.6909


grouped_std_mean = grouped_std_mean.round(2)
print(grouped_std_mean)

#summe_delta_s_quadrat = filterd['delta_s_quadrat'].sum()
#standartabweichung = math.sqrt(summe_delta_s_quadrat/199)
#print('standartabweichung: ' + str(standartabweichung))

# Formel t Verteilung: http://www.statistics4u.info/fundstat_germ/cc_confidence_interval.html
# T Verteilungs tabelle: https://www.staff.uni-mainz.de/klenke/vorlesungen/vorl_ws19-2/tabellenwerk.pdf
# Nachschauen:
# n -1 = 39 <- richtig?
# a = 0,005 bei 99,5% Wahrscheinlichkeit
# Werte 38-> 2,712 40-> 2,704 ~ 39 -> 2,7105
# neu: 39 -> 2.7079
# Formel:
# Erwartungswert +- Standartabweichung / sqrt(40) * 2,7105

   runtime ressource_limit       std
0  default         default  0.076268
1  default             max  0.078787
2  default             min  0.086499
3     kata         default  0.008396
4     kata             max  0.252319
5     kata             min  0.238438
6   katafc         default  0.009179
7   katafc             max  0.011020
8   katafc             min  0.013825
   runtime ressource_limit       mean
0  default         default  46.965314
1  default             max  46.963600
2  default             min   2.808080
3     kata         default  12.191743
4     kata             max  46.173286
5     kata             min   9.642674
6   katafc         default  12.027576
7   katafc             max  12.020206
8   katafc             min  12.018086
   runtime ressource_limit       std       mean
0  default         default  0.076268  46.965314
1  default             max  0.078787  46.963600
2  default             min  0.086499   2.808080
3     kata         default  0.008396  12.191743
4     kata

In [41]:
legend_dict = {'default': 'Docker (runc)', 'kata': 'Kata', 'katafc': 'Kata FC', 'runsc': 'gVisor (runsc)', 'runnc': 'Nabla (runnc)'} 

grouped_std_mean['runtime_legend'] = ''

for index, row in grouped_std_mean.iterrows():
    grouped_std_mean.loc[index, 'runtime_legend'] = legend_dict[row['runtime']]

In [49]:
######
# Leistung indivduell
####
source = ColumnDataSource(pd.DataFrame(grouped_std_mean))
#print(source.data)

runtimes = list(dict.fromkeys(source.data['runtime_legend'].tolist()))
#print(runtimes)

ressource_limits= list(dict.fromkeys(source.data['ressource_limit'].tolist()))
#print(ressource_limits)
limit_dict = {'default': 'default', 'max': 'maximalem', 'min': 'minimalem'}

for limit in ressource_limits:
    filterd_lmt=grouped_std_mean[grouped_std_mean['ressource_limit'] == limit]
    source = ColumnDataSource(pd.DataFrame(filterd_lmt))
    print(source.data)
    runtime_legend = source.data['runtime_legend'].tolist()
    
    palette = [bokeh.palettes.d3['Category10'][6][0], bokeh.palettes.d3['Category10'][6][2], bokeh.palettes.d3['Category10'][6][3]]
    color_map = factor_cmap(field_name='runtime_legend', palette=palette, factors=runtime_legend)
    

 
    p = figure(x_range=runtime, width=700, height=700, y_range=(0, 50), output_backend="svg")
    p.vbar(x='runtime_legend', top='mean', source=source, width=0.70, color=color_map)

    #p.add_layout(
    #  Whisker(source=source, base='runtime', upper="konf_intervall_oben", lower="konf_intervall_unten", level="overlay")
    #)

    p.title.text ='Rechenleistung mit ' + limit_dict[limit] + ' Ressourcenlimit ' 
    p.xaxis.axis_label = 'Runtime'
    p.yaxis.axis_label = 'Rechenleistung [GFlops]'
    p.yaxis.ticker = [0,10,20,30,40]
    
    p.title.text_font_size = '16pt'
    p.xaxis.axis_label_text_font_size = "15pt"
    p.xaxis.major_label_text_font_size = "13pt"

    p.yaxis.axis_label_text_font_size = "15pt"
    p.yaxis.major_label_text_font_size = "13pt"

    p.output_backend = "svg"


    output_notebook()

    show(p)

{'index': array([0, 3, 6]), 'runtime': array(['default', 'kata', 'katafc'], dtype=object), 'ressource_limit': array(['default', 'default', 'default'], dtype=object), 'std': array([0.08, 0.01, 0.01]), 'mean': array([46.97, 12.19, 12.03]), 'abweichung_in_prozent': array([0.05, 0.02, 0.02]), 'konf_intervall_unten': array([46.94, 12.19, 12.02]), 'konf_intervall_oben': array([46.99, 12.19, 12.03]), 'runtime_legend': array(['Docker (runc)', 'Kata', 'Kata FC'], dtype=object)}


Loading BokehJS ...

{'index': array([1, 4, 7]), 'runtime': array(['default', 'kata', 'katafc'], dtype=object), 'ressource_limit': array(['max', 'max', 'max'], dtype=object), 'std': array([0.08, 0.25, 0.01]), 'mean': array([46.96, 46.17, 12.02]), 'abweichung_in_prozent': array([0.05, 0.16, 0.03]), 'konf_intervall_unten': array([46.94, 46.1 , 12.02]), 'konf_intervall_oben': array([46.99, 46.25, 12.02]), 'runtime_legend': array(['Docker (runc)', 'Kata', 'Kata FC'], dtype=object)}


Loading BokehJS ...

{'index': array([2, 5, 8]), 'runtime': array(['default', 'kata', 'katafc'], dtype=object), 'ressource_limit': array(['min', 'min', 'min'], dtype=object), 'std': array([0.09, 0.24, 0.01]), 'mean': array([ 2.81,  9.64, 12.02]), 'abweichung_in_prozent': array([0.88, 0.71, 0.03]), 'konf_intervall_unten': array([ 2.78,  9.57, 12.01]), 'konf_intervall_oben': array([ 2.83,  9.71, 12.02]), 'runtime_legend': array(['Docker (runc)', 'Kata', 'Kata FC'], dtype=object)}


Loading BokehJS ...

In [5]:
print(grouped_std_mean.to_string())


   runtime ressource_limit   std   mean  send_abweichung_in_prozent  send_konf_intervall_unten  send_konf_intervall_oben
0  default         default  0.08  46.97                        0.05                      46.94                     46.99
1  default             max  0.08  46.96                        0.05                      46.94                     46.99
2  default             min  0.09   2.81                        0.88                       2.78                      2.83
3     kata         default  0.01  12.19                        0.02                      12.19                     12.19
4     kata             max  0.25  46.17                        0.16                      46.10                     46.25
5     kata             min  0.24   9.64                        0.71                       9.57                      9.71
6   katafc         default  0.01  12.03                        0.02                      12.02                     12.03
7   katafc             max  0.01

In [6]:
grouped_std_mean['docker_abweichung'] = 0

#print(grouped_std_mean.loc[ 273 , : ])

for index, row in grouped_std_mean.iterrows():
    send_docker_mean = grouped_std_mean.loc[(grouped_std_mean['runtime'] == 'default') & (grouped_std_mean['ressource_limit'] == row['ressource_limit']), ['mean']].reset_index().get_value(0, 'mean')
    #print(index)
    #print(row['mean'] * 100 / docker_mean)
    grouped_std_mean.loc[index, 'docker_abweichung'] = row['mean'] * 100 / send_docker_mean

grouped_std_mean_tcp = grouped_std_mean.round(2)

print(grouped_std_mean_tcp.to_string())
export_csv = grouped_std_mean_tcp.to_csv (r'Results/linpack_Gewichtung_Docker.csv', index = None, header=True, decimal=',')

   runtime ressource_limit   std   mean  send_abweichung_in_prozent  send_konf_intervall_unten  send_konf_intervall_oben  docker_abweichung
0  default         default  0.08  46.97                        0.05                      46.94                     46.99             100.00
1  default             max  0.08  46.96                        0.05                      46.94                     46.99             100.00
2  default             min  0.09   2.81                        0.88                       2.78                      2.83             100.00
3     kata         default  0.01  12.19                        0.02                      12.19                     12.19              25.95
4     kata             max  0.25  46.17                        0.16                      46.10                     46.25              98.32
5     kata             min  0.24   9.64                        0.71                       9.57                      9.71             343.06
6   katafc         d

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
